In [93]:
!pip install jsonpointer==2.3

In [94]:
!pip install jsonschema==4.17.3

In [95]:
!pip install openai==0.26.5

In [5]:
!export OPENAI_API_KEY=""

In [22]:
import json
import openai
import os

## Load config values

In [104]:

with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# Setting up the deployment name
deployment_name = config_details['COMPLETIONS_MODEL']

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
# openai.api_key = os.getenv("OPENAI_API_KEY")  
openai.api_key = ""

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Currently OPENAI API have the following versions available: 2022-12-01
openai.api_version = config_details['OPENAI_API_VERSION']

## Open the file containing the books reviews

In [1]:
with open('reviews.txt', 'r') as file:
    # Read contents of file
    reviews = file.read()
    # Print contents to console
    print(reviews)

## Creating a prompt and appending it to the reviews

In [2]:
# Give your prompt here
prompt = "Perform sentiment analysis on each of the following six book reviews. \n" \
"- Do not include author's name in the analysis \n" \
"- Provide overall sentiment score between 0 to 5 for the review \n" \
"- Mention the top positive aspects and top negative aspects, if any \n\n"

request = prompt + reviews + "\n\n"
print(request)

## Create a completion for the provided prompt and parameters. To know more about the parameters, checkout this documentation:  https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference

In [3]:
try:
    completion = openai.Completion.create(
                    prompt=request,
                    temperature=1,
                    max_tokens=1000,
                    top_p=0.5,
                    engine=deployment_name)

    # print the completion
    response = completion.choices[0].text.strip(" \n")
    print(response)
    
    
    # Here indicating if the response is filtered
    if completion.choices[0].finish_reason == "content_filter":
        print("The generated content is filtered.")
        
except openai.error.APIError as e:
    # Handle API error here, e.g. retry or log
    print(f"OpenAI API returned an API Error: {e}")

except openai.error.AuthenticationError as e:
    # Handle Authentication error here, e.g. invalid API key
    print(f"OpenAI API returned an Authentication Error: {e}")

except openai.error.APIConnectionError as e:
    # Handle connection error here
    print(f"Failed to connect to OpenAI API: {e}")

except openai.error.InvalidRequestError as e:
    # Handle connection error here
    print(f"Invalid Request Error: {e}")

except openai.error.RateLimitError as e:
    # Handle rate limit error
    print(f"OpenAI API request exceeded rate limit: {e}")

except openai.error.ServiceUnavailableError as e:
    # Handle Service Unavailable error
    print(f"Service Unavailable: {e}")

except openai.error.Timeout as e:
    # Handle request timeout
    print(f"Request timed out: {e}")

## Write the sentiments to a text file

In [119]:
with open("sentiments.txt", "w") as text_file:
    text_file.write(response)

In [6]:
with open('sentiments.txt', 'r') as file:
    # Read contents of file
    sentiments = file.read()
    # Print contents to console
    print(sentiments)

Book 1:
Overall sentiment score: 4.5
Positive aspects: Great walkthroughs and explanations on coding machine learning into different environment, good starter discussion on ML techniques, intuitive mx+b example of neural networks, answered lots of questions about using models, discussion of ML model deployment was all TensorFlow.
Negative aspects: Last chapter on ethics/federated learning felt a little short, could have more on dealing with/identifying model drift and training on new data.

Book 2:
Overall sentiment score: 4.5
Positive aspects: Perfect balance of breadth and depth into the topics, intuitive structure that elaborates at length on core ML concepts, does not overburden with complex programming.
Negative aspects: Figures/tables/code snippets in the book are grayscale.

Book 3:
Overall sentiment score: 4
Positive aspects: Fun and quick way of developing introduction to pytorch, appreciated the power of it, loved the graph neural networks chapter, introduces a topic and quic

## Creating a prompt and appending it to the sentiments

In [5]:
prompt = "Write a python code to read the following six books/records and create a pandas dataframe which has six rows and three columns (Overall sentiment score, Positive aspects, Negative aspect) and sort by Overall sentiment score \n\n"
request = prompt + sentiments
print(request)

In [130]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# Setting up the deployment name
deployment_name = config_details['CODEX_MODEL']

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
# openai.api_key = os.getenv("OPENAI_API_KEY")  
openai.api_key = ""

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Currently OPENAI API have the following versions available: 2022-12-01
openai.api_version = config_details['OPENAI_API_VERSION']

In [131]:
# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.Completion.create(
                    prompt=request,
                    temperature=1,
                    max_tokens=4000,
                    top_p=0.5,
                    engine=deployment_name)

# print the completion
response = completion.choices[0].text.strip(" \n")
print(response)

Note: Overall sentiment score is between 1 to 5.

Sample Output:

Overall sentiment score Positive aspects Negative aspect
5 Extremely gifted teacher with an uncanny ability to make optically complex tasks seem almost trivial, leverages Keras to demonstrate how some fairly astonishing things can happen with not that much code, should be required reading for anyone concerned with AI explainability. None.
4.5 Thorough breakdown surrounding the different data-oriented roles, clear responsibilities of each of the roles, good examples of where we can apply ML and the System Design breakdown, two end-to-end examples in the last two chapters. Could have some Q&A for the reader to test their knowledge or understanding of the concepts and when/where to apply them, could have the author's perspective on how they think the world of ML will change.
4.5 Perfect balance of breadth and depth into the topics, intuitive structure that elaborates at length on core ML concepts, does not overburden with c

In [133]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)
import pandas as pd
import numpy as np

df = pd.DataFrame({'Overall sentiment score':[4.5,4.5,4,2.5,5,4.5],
                   'Positive aspects':['Great walkthroughs and explanations on coding machine learning into different environment, good starter discussion on ML techniques, intuitive mx+b example of neural networks, answered lots of questions about using models, discussion of ML model deployment was all TensorFlow.',
                                      'Perfect balance of breadth and depth into the topics, intuitive structure that elaborates at length on core ML concepts, does not overburden with complex programming.',
                                      'Fun and quick way of developing introduction to pytorch, appreciated the power of it, loved the graph neural networks chapter, introduces a topic and quick math and immediate coding.',
                                      'Good intro for student of AI and machine learning, helps you get a grasp on machine learning quickly, author does not always hold you hand through the whole journey.',
                                      'Extremely gifted teacher with an uncanny ability to make optically complex tasks seem almost trivial, leverages Keras to demonstrate how some fairly astonishing things can happen with not that much code, should be required reading for anyone concerned with AI explainability.',
                                      'Thorough breakdown surrounding the different data-oriented roles, clear responsibilities of each of the roles, good examples of where we can apply ML and the System Design breakdown, two end-to-end examples in the last two chapters.'],
                   'Negative aspect':['Last chapter on ethics/federated learning felt a little short, could have more on dealing with/identifying model drift and training on new data.',
                                     'Figures/tables/code snippets in the book are grayscale.',
                                     'Missed GANs and timeseries applications, initial chapters were all about classical ML, the treatment of the subject is pretty similar to most books in the market.',
                                     'Sample codes in the book start to misalign with the sample codes given in the GitHub, not properly tested, some of the issues have been left unattended for months, author does not spend a whole lot of time on explaining the details of fundamental AI/ML concepts.',
                                     'None.',
                                     'Could have some Q&A for the reader to test their knowledge or understanding of the concepts and when/where to apply them, could have the author\'s perspective on how they think the world of ML will change.']})

df = df.sort_values(by=['Overall sentiment score'], ascending=False)
df


,Overall sentiment score,Positive aspects,Negative aspect
4,5.0,Extremely gifted teacher with an uncanny ability to make optically complex tasks seem almost tri...,None.
0,4.5,"Great walkthroughs and explanations on coding machine learning into different environment, good ...","Last chapter on ethics/federated learning felt a little short, could have more on dealing with/i..."
1,4.5,"Perfect balance of breadth and depth into the topics, intuitive structure that elaborates at len...",Figures/tables/code snippets in the book are grayscale.
5,4.5,"Thorough breakdown surrounding the different data-oriented roles, clear responsibilities of each...",Could have some Q&A for the reader to test their knowledge or understanding of the concepts and ...
2,4.0,"Fun and quick way of developing introduction to pytorch, appreciated the power of it, loved the ...","Missed GANs and timeseries applications, initial chapters were all about classical ML, the treat..."
3,2.5,"Good intro for student of AI and machine learning, helps you get a grasp on machine learning qui...","Sample codes in the book start to misalign with the sample codes given in the GitHub, not proper..."
